In [1]:
import boto3
import time
import json
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

from src.data_reader import DataReader, RetractionFinder
from src.to_gexf import to_gexf
from src.get_redacted import get_paper, load_redacted, get_doi, gen_retracted
import src.load_data as load_data

import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import dataframe
import graphframes


In [2]:
client = MongoClient()
db = client['capstone']
papers = db.papers
s3 = boto3.client('s3')

In [3]:
def write_database(coll, start=0, end=39):
    tot_lines = 0
    for i in range(start, end + 1):
        t = time.process_time()
        corpus = s3.get_object(
            Bucket='alexklein', 
            Key=f'capstone/data/s2-corpus-{i if i >= 10 else "0" + str(i)}')
        for line in corpus['Body'].iter_lines():
            entry = json.loads(line)
            entry['_id'] = entry['id']
            del entry['id']
            try:
                coll.insert_one(document=entry)
            except DuplicateKeyError:
                continue
        print(f's2-corpus-{i if i >= 10 else "0" + str(i)} completed in {(time.process_time()-t):0.0f} s.')
        print(f'Estimated document count at {coll.estimated_document_count(maxTimeMS=50)}')



In [20]:
def construct_component(collection, start_id=None, 
                        max_depth=5, max_size=1e6,
                        component=set(), depth=0, v=False):
    if start_id in component or depth > max_depth and max_depth > 0 \
        or len(component) > max_size: 
        return component
    component |= {start_id}
    node = collection.find_one(start_id)
    if node is None: 
        return component
    for other in node['inCitations'] + node['outCitations']:
        component = construct_component(collection, other, 
                                        max_depth, component, 
                                        depth=depth + 1, v=v)
    if v:
        print(len(component), depth)
    return component

In [ ]:
def construct_component(collection, start_id=None, 
                        max_depth=5, component=set(), 
                        depth=0, v=False):
    if start_id in component or depth > max_depth and max_depth > 0: 
        return component
    component |= {start_id}
    node = collection.find_one(start_id)
    if node is None: 
        return component
    for other in node['inCitations'] + node['outCitations']:
        component = construct_component(collection, other, 
                                        max_depth, component, 
                                        depth=depth + 1, v=v)
    if v:
        print(len(component), depth)
    return component

In [5]:
def write_component(collection, component, path='data/component'):
    result = collection.find({'_id': {'$in': list(component)}})
    with open(path, 'w') as f:
        for itm in result:
            f.writelines(json.dumps(itm))
            f.writelines('\n')

In [21]:
comp = construct_component(papers, 
                    '7b800bf310846134fb4a19b2191612cfe96974ee', 
                    v=False)


5 4
6 4
7 4
8 4
9 4
10 4
11 4
12 4
13 4
14 4
15 4
16 4
17 4
18 4
19 4
20 4
20 3
22 4
22 3
24 4
25 4
26 4
27 4
28 4
29 4
30 4
31 4
32 4
33 4
34 4
35 4
36 4
37 4
38 4
39 4
40 4
41 4
42 4
43 4
44 4
45 4
46 4
47 4
48 4
49 4
50 4
51 4
52 4
53 4
54 4
55 4
56 4
57 4
58 4
59 4
60 4
61 4
62 4
63 4
64 4
65 4
66 4
67 4
68 4
69 4
70 4
71 4
72 4
73 4
73 3
75 4
76 4
77 4
78 4
79 4
80 4
81 4
82 4
83 4
84 4
85 4
85 3
87 4
88 4
89 4
90 4
91 4
92 4
93 4
94 4
95 4
96 4
97 4
98 4
99 4
100 4
101 4
102 4
103 4
104 4
105 4
106 4
107 4
108 4
109 4
110 4
111 4
112 4
113 4
114 4
115 4
116 4
117 4
118 4
119 4
120 4
121 4
122 4
123 4
124 4
125 4
126 4
127 4
128 4
129 4
130 4
131 4
132 4
133 4
134 4
135 4
136 4
137 4
138 4
139 4
140 4
141 4
142 4
143 4
144 4
145 4
146 4
147 4
148 4
149 4
150 4
151 4
152 4
153 4
154 4
155 4
156 4
157 4
158 4
159 4
160 4
161 4
162 4
163 4
164 4
165 4
166 4
167 4
168 4
169 4
170 4
171 4
172 4
173 4
174 4
175 4
176 4
177 4
178 4
179 4
180 4
181 4
182 4
183 4
184 4
185 4
186 4
187 4
18

1409 4
1410 4
1411 4
1412 4
1413 4
1414 4
1415 4
1416 4
1417 4
1418 4
1419 4
1420 4
1421 4
1422 4
1423 4
1424 4
1425 4
1426 4
1427 4
1428 4
1429 4
1430 4
1431 4
1432 4
1433 4
1434 4
1435 4
1436 4
1437 4
1438 4
1439 4
1440 4
1441 4
1442 4
1443 4
1444 4
1445 4
1446 4
1447 4
1448 4
1449 4
1450 4
1451 4
1452 4
1453 4
1454 4
1455 4
1456 4
1457 4
1458 4
1459 4
1460 4
1461 4
1462 4
1463 4
1464 4
1465 4
1466 4
1467 4
1468 4
1469 4
1470 4
1471 4
1472 4
1473 4
1474 4
1475 4
1476 4
1477 4
1478 4
1479 4
1480 4
1481 4
1482 4
1483 4
1484 4
1485 4
1486 4
1487 4
1488 4
1489 4
1490 4
1491 4
1492 4
1493 4
1494 4
1495 4
1496 4
1497 4
1498 4
1499 4
1500 4
1501 4
1502 4
1503 4
1504 4
1505 4
1506 4
1507 4
1508 4
1509 4
1510 4
1511 4
1512 4
1513 4
1514 4
1515 4
1516 4
1517 4
1518 4
1519 4
1520 4
1521 4
1522 4
1523 4
1524 4
1525 4
1526 4
1527 4
1528 4
1529 4
1530 4
1531 4
1532 4
1533 4
1534 4
1535 4
1536 4
1537 4
1538 4
1539 4
1540 4
1541 4
1542 4
1543 4
1544 4
1545 4
1546 4
1547 4
1548 4
1549 4
1550 4
1551 4

2763 4
2764 4
2765 4
2766 4
2767 4
2768 4
2769 4
2770 4
2771 4
2772 4
2773 4
2774 4
2774 3
2776 4
2777 4
2778 4
2779 4
2780 4
2781 4
2782 4
2783 4
2783 3
2785 4
2786 4
2787 4
2788 4
2789 4
2790 4
2791 4
2792 4
2793 4
2794 4
2795 4
2796 4
2797 4
2798 4
2799 4
2800 4
2801 4
2802 4
2803 4
2803 3
2805 4
2806 4
2807 4
2808 4
2809 4
2810 4
2811 4
2812 4
2813 4
2814 4
2815 4
2816 4
2817 4
2818 4
2819 4
2820 4
2821 4
2822 4
2823 4
2824 4
2825 4
2826 4
2827 4
2828 4
2829 4
2830 4
2831 4
2832 4
2833 4
2834 4
2835 4
2836 4
2837 4
2838 4
2839 4
2840 4
2841 4
2842 4
2843 4
2844 4
2845 4
2846 4
2847 4
2848 4
2849 4
2850 4
2851 4
2852 4
2853 4
2854 4
2855 4
2856 4
2857 4
2858 4
2859 4
2860 4
2861 4
2862 4
2863 4
2864 4
2865 4
2866 4
2867 4
2868 4
2869 4
2870 4
2870 3
2872 4
2873 4
2874 4
2875 4
2876 4
2877 4
2878 4
2879 4
2880 4
2881 4
2882 4
2883 4
2883 3
2885 4
2886 4
2887 4
2888 4
2889 4
2890 4
2890 3
2892 4
2893 4
2894 4
2895 4
2896 4
2897 4
2898 4
2899 4
2900 4
2900 3
2902 4
2903 4
2904 4
2905 4

4014 4
4015 4
4016 4
4017 4
4018 4
4019 4
4020 4
4021 4
4022 4
4023 4
4024 4
4025 4
4026 4
4027 4
4028 4
4029 4
4030 4
4031 4
4032 4
4033 4
4034 4
4035 4
4036 4
4037 4
4038 4
4039 4
4040 4
4041 4
4042 4
4043 4
4044 4
4045 4
4046 4
4047 4
4048 4
4049 4
4050 4
4051 4
4052 4
4053 4
4054 4
4055 4
4056 4
4057 4
4058 4
4059 4
4060 4
4060 3
4062 4
4063 4
4064 4
4065 4
4066 4
4067 4
4068 4
4069 4
4070 4
4071 4
4072 4
4073 4
4074 4
4075 4
4076 4
4077 4
4078 4
4079 4
4080 4
4081 4
4082 4
4083 4
4084 4
4085 4
4086 4
4087 4
4088 4
4089 4
4090 4
4091 4
4092 4
4093 4
4094 4
4095 4
4096 4
4097 4
4098 4
4099 4
4100 4
4101 4
4102 4
4103 4
4104 4
4105 4
4106 4
4107 4
4108 4
4109 4
4110 4
4111 4
4112 4
4113 4
4114 4
4115 4
4116 4
4117 4
4118 4
4119 4
4120 4
4121 4
4122 4
4123 4
4124 4
4125 4
4126 4
4127 4
4128 4
4129 4
4130 4
4131 4
4132 4
4133 4
4134 4
4135 4
4136 4
4137 4
4138 4
4139 4
4140 4
4141 4
4142 4
4143 4
4144 4
4145 4
4146 4
4147 4
4148 4
4149 4
4150 4
4151 4
4152 4
4153 4
4154 4
4155 4
4156 4

5332 4
5333 4
5334 4
5335 4
5336 4
5337 4
5338 4
5338 3
5340 4
5341 4
5342 4
5343 4
5344 4
5345 4
5346 4
5347 4
5348 4
5349 4
5350 4
5351 4
5351 3
5353 4
5354 4
5355 4
5356 4
5357 4
5358 4
5359 4
5360 4
5361 4
5362 4
5363 4
5364 4
5365 4
5366 4
5367 4
5368 4
5369 4
5370 4
5370 3
5372 4
5373 4
5374 4
5375 4
5376 4
5377 4
5378 4
5379 4
5380 4
5381 4
5382 4
5383 4
5384 4
5385 4
5386 4
5387 4
5388 4
5389 4
5390 4
5391 4
5392 4
5393 4
5394 4
5395 4
5396 4
5397 4
5398 4
5398 3
5400 4
5401 4
5402 4
5403 4
5404 4
5404 3
5406 4
5407 4
5408 4
5409 4
5410 4
5410 3
5412 4
5413 4
5414 4
5415 4
5416 4
5417 4
5418 4
5418 3
5420 4
5421 4
5422 4
5423 4
5424 4
5425 4
5426 4
5427 4
5428 4
5429 4
5430 4
5431 4
5432 4
5433 4
5434 4
5435 4
5436 4
5437 4
5438 4
5439 4
5440 4
5441 4
5441 3
5443 4
5444 4
5445 4
5446 4
5447 4
5448 4
5449 4
5450 4
5451 4
5452 4
5453 4
5454 4
5455 4
5456 4
5457 4
5458 4
5459 4
5460 4
5461 4
5462 4
5463 4
5464 4
5465 4
5466 4
5467 4
5468 4
5469 4
5470 4
5471 4
5472 4
5473 4
5474 4

...

63723 4
63724 4
63725 4
63726 4
63727 4
63728 4
63729 4
63730 4
63731 4
63732 4
63733 4
63734 4
63735 4
63736 4
63737 4
63738 4
63739 4
63740 4
63741 4
63742 4
63743 4
63744 4
63745 4
63746 4
63747 4
63748 4
63749 4
63750 4
63751 4
63752 4
63753 4
63754 4
63755 4
63756 4
63757 4
63758 4
63759 4
63760 4
63761 4
63762 4
63763 4
63764 4
63765 4
63766 4
63767 4
63768 4
63769 4
63770 4
63771 4
63772 4
63773 4
63774 4
63775 4
63776 4
63777 4
63778 4
63779 4
63780 4
63781 4
63782 4
63783 4
63784 4
63785 4
63786 4
63787 4
63788 4
63789 4
63790 4
63791 4
63792 4
63793 4
63794 4
63795 4
63796 4
63797 4
63798 4
63799 4
63800 4
63801 4
63802 4
63803 4
63804 4
63805 4
63806 4
63807 4
63808 4
63809 4
63810 4
63811 4
63812 4
63813 4
63814 4
63815 4
63816 4
63817 4
63818 4
63819 4
63820 4
63821 4
63822 4
63823 4
63824 4
63825 4
63826 4
63827 4
63828 4
63829 4
63830 4
63831 4
63832 4
63833 4
63834 4
63835 4
63836 4
63837 4
63838 4
63839 4
63840 4
63841 4
63842 4
63843 4
63844 4
63845 4
63846 4
63847 4


64924 4
64925 4
64926 4
64927 4
64928 4
64929 4
64930 4
64931 4
64932 4
64933 4
64934 4
64935 4
64936 4
64937 4
64938 4
64939 4
64940 4
64941 4
64942 4
64943 4
64944 4
64945 4
64946 4
64947 4
64948 4
64949 4
64950 4
64951 4
64952 4
64953 4
64954 4
64955 4
64956 4
64957 4
64958 4
64959 4
64960 4
64961 4
64962 4
64962 3
64964 4
64965 4
64966 4
64967 4
64968 4
64969 4
64970 4
64971 4
64972 4
64973 4
64974 4
64975 4
64976 4
64977 4
64978 4
64979 4
64980 4
64981 4
64982 4
64983 4
64984 4
64985 4
64986 4
64987 4
64988 4
64989 4
64990 4
64990 3
64992 4
64993 4
64994 4
64995 4
64996 4
64997 4
64998 4
64999 4
65000 4
65001 4
65002 4
65003 4
65004 4
65005 4
65006 4
65007 4
65008 4
65009 4
65010 4
65011 4
65012 4
65013 4
65014 4
65015 4
65016 4
65017 4
65018 4
65019 4
65020 4
65021 4
65022 4
65023 4
65024 4
65025 4
65026 4
65027 4
65028 4
65029 4
65030 4
65031 4
65032 4
65033 4
65034 4
65035 4
65036 4
65037 4
65038 4
65039 4
65040 4
65041 4
65042 4
65043 4
65044 4
65045 4
65046 4
65047 4
65048 4


66109 4
66110 4
66111 4
66112 4
66113 4
66114 4
66115 4
66116 4
66117 4
66118 4
66119 4
66120 4
66121 4
66122 4
66123 4
66124 4
66125 4
66126 4
66127 4
66128 4
66129 4
66130 4
66131 4
66132 4
66133 4
66134 4
66135 4
66136 4
66137 4
66138 4
66139 4
66140 4
66141 4
66142 4
66143 4
66144 4
66145 4
66146 4
66147 4
66148 4
66149 4
66150 4
66151 4
66152 4
66153 4
66154 4
66155 4
66156 4
66157 4
66158 4
66159 4
66160 4
66161 4
66162 4
66163 4
66164 4
66165 4
66166 4
66167 4
66168 4
66169 4
66170 4
66171 4
66172 4
66173 4
66174 4
66175 4
66176 4
66177 4
66178 4
66179 4
66180 4
66181 4
66182 4
66183 4
66184 4
66185 4
66186 4
66187 4
66188 4
66189 4
66190 4
66191 4
66192 4
66193 4
66194 4
66195 4
66196 4
66197 4
66198 4
66198 3
66200 4
66201 4
66202 4
66203 4
66204 4
66205 4
66206 4
66207 4
66208 4
66209 4
66210 4
66211 4
66211 3
66213 4
66214 4
66215 4
66216 4
66217 4
66218 4
66219 4
66220 4
66221 4
66222 4
66223 4
66224 4
66225 4
66226 4
66227 4
66228 4
66229 4
66230 4
66231 4
66232 4
66233 4


67280 4
67281 4
67282 4
67283 4
67284 4
67285 4
67286 4
67287 4
67288 4
67289 4
67290 4
67291 4
67292 4
67293 4
67294 4
67295 4
67296 4
67297 4
67298 4
67299 4
67300 4
67301 4
67302 4
67303 4
67304 4
67305 4
67306 4
67307 4
67308 4
67309 4
67310 4
67311 4
67312 4
67313 4
67314 4
67315 4
67316 4
67317 4
67318 4
67319 4
67320 4
67321 4
67322 4
67323 4
67324 4
67325 4
67326 4
67327 4
67328 4
67329 4
67330 4
67331 4
67332 4
67333 4
67334 4
67335 4
67336 4
67337 4
67338 4
67339 4
67340 4
67341 4
67342 4
67343 4
67344 4
67345 4
67346 4
67347 4
67348 4
67349 4
67350 4
67351 4
67352 4
67353 4
67354 4
67355 4
67356 4
67357 4
67358 4
67359 4
67360 4
67361 4
67362 4
67363 4
67364 4
67365 4
67366 4
67367 4
67368 4
67369 4
67370 4
67371 4
67372 4
67373 4
67374 4
67375 4
67376 4
67377 4
67378 4
67379 4
67380 4
67381 4
67382 4
67383 4
67384 4
67385 4
67386 4
67387 4
67388 4
67389 4
67390 4
67391 4
67392 4
67393 4
67394 4
67395 4
67396 4
67397 4
67398 4
67399 4
67400 4
67401 4
67402 4
67403 4
67404 4


68492 4
68493 4
68494 4
68495 4
68496 4
68497 4
68498 4
68499 4
68500 4
68501 4
68502 4
68503 4
68504 4
68505 4
68506 4
68507 4
68508 4
68509 4
68510 4
68511 4
68512 4
68513 4
68514 4
68515 4
68516 4
68517 4
68518 4
68519 4
68520 4
68521 4
68522 4
68523 4
68524 4
68525 4
68526 4
68527 4
68528 4
68529 4
68530 4
68531 4
68532 4
68533 4
68534 4
68535 4
68536 4
68537 4
68538 4
68539 4
68540 4
68541 4
68542 4
68543 4
68544 4
68545 4
68546 4
68547 4
68548 4
68549 4
68550 4
68551 4
68552 4
68553 4
68554 4
68555 4
68556 4
68557 4
68558 4
68559 4
68560 4
68561 4
68562 4
68563 4
68564 4
68565 4
68566 4
68567 4
68568 4
68569 4
68570 4
68571 4
68572 4
68573 4
68573 3
68575 4
68576 4
68577 4
68578 4
68579 4
68580 4
68581 4
68582 4
68583 4
68584 4
68585 4
68586 4
68587 4
68588 4
68589 4
68590 4
68591 4
68592 4
68593 4
68594 4
68595 4
68596 4
68597 4
68598 4
68599 4
68600 4
68601 4
68602 4
68603 4
68604 4
68605 4
68606 4
68607 4
68608 4
68609 4
68610 4
68611 4
68612 4
68613 4
68614 4
68615 4
68616 4


In [22]:
write_component(papers, comp)

In [5]:
reader = DataReader('data/component')
reader.write(dynamic=True)

In [6]:
edges = spark.createDataFrame(reader.read_edges(), schema=StructType(
[
    StructField(name='src', dataType=StringType()),
    StructField(name='dst', dataType=StringType()),
    StructField(name='year', dataType=StringType())
]
)).withColumn('id', F.monotonically_increasing_id())


nodes = spark.createDataFrame(reader.read_nodes(), schema=StructType(
[
    StructField(name='id', dataType=StringType()),
    StructField(name='year', dataType=StringType()),
    StructField(name='authors', dataType=StringType())
]
))

In [8]:
g = graphframes.GraphFrame(e=edges, v=nodes)
to_gexf(g, 'data/component.gexf', dynamic=True)

In [ ]:
def to_gexf(edges_path, node_path, path, dynamic=False):
    """
    Export a pyspark graphframe to a .gexf file for use in Gephi.
    ============
    Takes:
        g: pyspark graphframe object-graph to export
        path: string-location to save file
    ============
    Returns: None
    """
    if dynamic:
        with open(nodes_path, 'r') as f:
            
            nodes = ''.join(g.vertices.rdd.map(lambda v: f'      <node id="{v["id"]}"  start="{v["year"]}" end="2018" />\n').collect())
                
        
        edges = ''.join(g.edges.rdd.map(lambda e: f'      <edge id="{e["id"]}" source="{e["src"]}" target="{e["dst"]}" start="{e["year"]}" end="2018" />\n').collect())
    else:
      nodes = ''.join(g.vertices.rdd.map(lambda v: f'      <node id="{v["id"]}" />\n').collect())
      edges = ''.join(g.edges.rdd.map(lambda e: f'      <edge id="{e["id"]}" source="{e["src"]}" target="{e["dst"]}" />\n').collect())
    string =f"""<?xml version="1.0" encoding="UTF-8"?>
    <gexf xmlns="http://www.gexf.net/1.2draft" version="1.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/XMLSchema-instance">
      <graph mode="{'static' if not dynamic else 'dynamic'}" defaultedgetype="directed" name="">
        <nodes>
  {nodes}
        </nodes>
        <edges>
  {edges}
        </edges>
      </graph>
    </gexf>"""
    with open(path, 'w') as f:
        f.write(string)